<a href="https://colab.research.google.com/github/dannyoo/Data-Analytics-Playground/blob/master/Loan_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


#Linear Regression

For linear regression, goal is to predict the Interest rate

In [0]:
li = pd.read_csv('https://spark-public.s3.amazonaws.com/dataanalysis/loansData.csv')
li.head()

,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
81174,20000,20000.0,8.90%,36 months,debt_consolidation,14.90%,SC,MORTGAGE,6541.67,735-739,14.0,14272.0,2.0,< 1 year
99592,19200,19200.0,12.12%,36 months,debt_consolidation,28.36%,TX,MORTGAGE,4583.33,715-719,12.0,11140.0,1.0,2 years
80059,35000,35000.0,21.98%,60 months,debt_consolidation,23.81%,CA,MORTGAGE,11500.00,690-694,14.0,21977.0,1.0,2 years
15825,10000,9975.0,9.99%,36 months,debt_consolidation,14.30%,KS,MORTGAGE,3833.33,695-699,10.0,9346.0,0.0,5 years
33182,12000,12000.0,11.71%,36 months,credit_card,18.78%,NJ,RENT,3195.00,695-699,11.0,14469.0,0.0,9 years


In [0]:
li['Interest.Rate'] = [float(i[:-1]) for i in li['Interest.Rate']]

In [0]:
li['Loan.Length']= [float(i.split(" ")[0]) for i in li['Loan.Length']]

In [0]:
li['FICO.Range'] = [(float(i.split("-")[0])+float(i.split("-")[1]))/2 for i in li['FICO.Range']]

In [0]:
df = li[['Interest.Rate', 'Loan.Length', 'FICO.Range']]

In [0]:
intrate = df['Interest.Rate']
loanamt = df['Loan.Length']
fico = df['FICO.Range']

# reshape the data from a pandas Series to columns 
# the dependent variable
y = np.array(intrate).transpose()
# the independent variables shaped as columns
x1 = np.array(fico).transpose()
x2 = np.array(loanamt).transpose()

# put the two columns together to create an input matrix 
# if we had n independent variables we would have n columns here
x = np.column_stack([x1,x2])

# create a linear model and fit it to the data
X = sm.add_constant(x)
model = sm.OLS(y,X)
f = model.fit()

print('Coefficients: ', f.params[0:2])
print('Intercept: ', f.params[2])
print('P-Values: ', f.pvalues)
print('R-Squared: ', f.rsquared)

Coefficients:  [65.8744788  -0.08520339]
Intercept:  0.18193967135306596
P-Values:  [0.00000000e+000 0.00000000e+000 4.27159025e-258]
R-Squared:  0.6898129718512984


#Logistic Regression

For logistic regression, goal is to predict the probability of getting a loan

In [0]:
lo = pd.read_csv('https://raw.githubusercontent.com/nborwankar/LearnDataScience/master/datasets/loanf.csv')
lo.tail()

,Interest.Rate,FICO.Score,Loan.Length,Monthly.Income,Loan.Amount
2472,7.90,770,36,5166.67,19000
2481,6.03,760,36,5000.00,10000
2482,6.62,810,36,9250.00,27000
2486,10.75,765,36,7083.33,25000
2497,14.09,740,60,8903.25,16000


In [0]:
lo['TF']=lo['Interest.Rate']<=12
lo.tail()

,Interest.Rate,FICO.Score,Loan.Length,Monthly.Income,Loan.Amount,TF
2472,7.90,770,36,5166.67,19000,True
2481,6.03,760,36,5000.00,10000,True
2482,6.62,810,36,9250.00,27000,True
2486,10.75,765,36,7083.33,25000,True
2497,14.09,740,60,8903.25,16000,False


In [0]:
# statsmodels requires us to add a constant column representing the intercept
lo['intercept']=1.0
# identify the independent variables 
ind_cols=['FICO.Score','Loan.Amount','intercept']
logit = sm.Logit(lo['TF'], lo[ind_cols])
result=logit.fit()

Optimization terminated successfully.
         Current function value: 0.319503
         Iterations 8


In [0]:
coeff = result.params
print(coeff)

FICO.Score      0.087423
Loan.Amount    -0.000174
intercept     -60.125045
dtype: float64


In [0]:
import math

In [0]:
def pz(fico,amt,coeff):
  # compute the linear expression by multipyling the inputs by their respective coefficients.
  # note that the coefficient array has the intercept coefficient at the end
  z = coeff[0]*fico + coeff[1]*amt + coeff[2]
  return 1/(1+math.exp(-1*z))

In [0]:
pz(800, 50000, coeff)

0.7525257852724159